In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import parselmouth
from scipy.fftpack import fft
from scipy.signal import welch
from scipy.signal import lfilter
from scipy.signal import hamming
from scipy.signal import convolve
import matplotlib.pyplot as plt
from scipy.signal import get_window

In [2]:
def extract_pitch(filename):
    # Load data from the .dat file into a numpy array
    data = np.fromfile(filename, dtype=np.int16)
    sr = 8000  # the sample rate is 8kHz

    # Convert the data to floating-point
    data = data.astype(np.float32) / 32768

    # Compute the fundamental frequency (pitch)
    pitch, mag = librosa.piptrack(data, sr=sr)
    pitch = pitch[mag > np.median(mag)]

    return pitch

In [3]:
def extract_jitter(pitches):

    # Calculate the differences between consecutive frames
    differences = np.diff(pitches)

    # Calculate the absolute differences
    absolute_differences = np.abs(differences)

    # Calculate the jitter by taking the standard deviation of the absolute differences
    jitter = np.std(absolute_differences)

    return jitter

In [4]:
def calculate_rap(pitch_periods):
    # Calculate the differences between consecutive pitch periods
    differences = np.diff(pitch_periods)

    # Calculate the absolute differences, relative to the average pitch period
    abs_differences = np.abs(differences / ((pitch_periods[:-1] + pitch_periods[1:]) / 2))

    # Calculate the RAP as the mean of the absolute differences, expressed as a percentage
    rap = np.mean(abs_differences) * 100

    return rap

In [5]:
def pitch_period_perturbation_quotient(file_path):
    # Load the data from .dat file
    data = np.fromfile(file_path, dtype=np.int16)

    # Set the parameters for the analysis
    frame_length = int(0.03 * 8000)  # 30 ms
    step_size = int(0.01 * 8000)  # 10 ms
    window = get_window('hamming', frame_length)

    # Initialize the arrays to store pitch periods and pitch period perturbations
    pitch_periods = []
    pitch_period_perturbations = []

    # Iterate over the frames
    for i in range(0, len(data) - frame_length, step_size):
        # Extract the current frame
        frame = data[i:i + frame_length]

        # Apply the window to the frame
        windowed_frame = window * frame

        # Compute the power spectrum of the windowed frame
        power_spectrum = np.abs(fft(windowed_frame))**2

        # Find the fundamental frequency by locating the peak in the power spectrum
        fundamental_frequency_index = np.argmax(power_spectrum[0:int(frame_length / 2)])
        fundamental_frequency = fundamental_frequency_index * 8000 / frame_length

        # Compute the pitch period
        if fundamental_frequency!=0:
            pitch_period = 1 / fundamental_frequency
            pitch_periods.append(pitch_period)

    # Compute the pitch period perturbation quotient
    pitch_period_perturbation_quotient = (np.max(pitch_periods) - np.min(pitch_periods)) / np.mean(pitch_periods)

    return pitch_period_perturbation_quotient

In [6]:
def smoothed_pitch_period_perturbation_quotient(file_path):
    # Load the speech signal from .dat file
    signal = np.fromfile(file_path, dtype='float32')

    # Reshape the signal into a 2D array
    nframes = int(signal.shape[0] / 80)
    signal = signal[:nframes*80]
    signal = np.reshape(signal, (nframes, 80))

    # Compute the pitch period using autocorrelation method
    pitch_periods = []
    for i in range(nframes):
        acf = np.correlate(signal[i], signal[i], mode='full')
        acf = acf[len(acf)//2:]
        peak = np.argmax(acf[5:40]) + 5
        pitch_period = (peak / 8000) * 1000  # Convert to milliseconds
        pitch_periods.append(pitch_period)

    # Compute the smoothed pitch period using a median filter
    smoothed_pitch_periods = np.median([pitch_periods[i-1:i+2] for i in range(1, len(pitch_periods)-1)], axis=1)

    # Compute the SPPQ
    num_periods = len(smoothed_pitch_periods)
    sppq = np.sqrt(np.sum(np.square((smoothed_pitch_periods[1:num_periods-1] - smoothed_pitch_periods[0:num_periods-2]) - (smoothed_pitch_periods[2:num_periods] - smoothed_pitch_periods[1:num_periods-1]))) / (2 * (num_periods - 2))) * 100

    return sppq

In [7]:
def calculate_f0_variation(file_path):
    # Load the DAT file
    audio = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    audio = audio / np.max(np.abs(audio))

    # Define the sampling frequency
    fs = 8000

    # Define the window size and hop size
    window_size = int(0.03 * fs)  # 30 ms
    hop_size = int(0.01 * fs)  # 10 ms

    # Calculate the pitch using autocorrelation method
    pitch_values = []
    for i in range(0, len(audio) - window_size, hop_size):
        window = audio[i:i + window_size]
        autocorr = np.correlate(window, window, mode='full')
        if np.argmax(autocorr[window_size:])>0:
            pitch = fs / np.argmax(autocorr[window_size:])  # Calculate the pitch
            pitch_values.append(pitch)

    # Calculate the fundamental frequency variation
    f0_variation = np.std(pitch_values) / np.mean(pitch_values)
    return f0_variation

In [8]:
def shimmer(file_path):
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    sample_rate = 8000
    
    # Set the pitch period to 5 ms (200 Hz)
    pitch_period = int(sample_rate * 0.005)

    # extract pitch periods
    pitch_periods = []
    for i in range(0, len(signal)-pitch_period, pitch_period):
        pitch_periods.append(signal[i:i+pitch_period])

    # calculate shimmer
    max_amps = np.max(pitch_periods, axis=1)
    min_amps = np.min(pitch_periods, axis=1)
    shimmer_vals = max_amps / min_amps
    return np.mean(shimmer_vals)

In [9]:
def relative_standard_dev_voice_amp(file_path):

    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    sample_rate = 8000

    # convert the signal to floating-point values
    signal = signal.astype(np.float)

    # calculate the relative standard deviation of the signal amplitude
    rel_std = 100 * np.std(signal) / np.mean(signal)
    
    return rel_std

In [16]:
def amplitude_perturbation_quotient(file_path):
    
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    fs = 8000
    
    # Center the signal at 0
    signal = signal - np.mean(signal)
    
    # Compute the envelope of the signal
    envelope = np.abs(signal)
    
    # Divide the signal into 10ms windows
    window_size = int(fs / 100)
    windows = np.array([envelope[i:i+window_size] for i in range(0, len(envelope) - window_size, window_size)])
    
    # Compute the standard deviation of each window
    window_sd = np.std(windows, axis=1)
    
    # Compute the mean of the window standard deviations
    sd_mean = np.mean(window_sd)
    
    # Compute the amplitude perturbation quotient
    apq = (100 * sd_mean) / np.mean(envelope)
    
    return apq

In [11]:
def smoothed_amplitude_perturbation_quotient(file_path):
    
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    sample_rate = 8000
    
    # Calculate the autocorrelation of the signal
    corr = np.correlate(signal, signal, mode='full')

    # Find the index of the first peak in the autocorrelation
    start = int(np.floor(sample_rate / 500)) # Start search at 2 ms
    end = int(np.floor(sample_rate / 75)) # End search at 13 ms
    peak = start + np.argmax(corr[start:end])

    # Compute the pitch period
    period = (peak / sample_rate) * 1000

    # Compute the amplitude of each period
    amplitude = np.zeros_like(signal)
    for i in range(len(signal)):
        if i % peak == 0:
            amplitude[i] = np.abs(signal[i])
            
    # Apply a moving average filter to the amplitude of the speech signal
    smoothed_amplitude = np.zeros_like(amplitude)
    for i in range(1, len(amplitude)-1):
        smoothed_amplitude[i] = np.mean(amplitude[i-1:i+2])

    # Compute the SAPQ
    sapq = []
    for i in range(0,len(amplitude)):
        if (amplitude[i] + smoothed_amplitude[i])!=0: 
            sapq.append(np.abs(amplitude[i] - smoothed_amplitude[i]) / (0.5 * (amplitude[i] + smoothed_amplitude[i])))

    return 100*np.mean(sapq)

In [12]:
def noise_to_harmonic_ratio(file_path):
    
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    fs = 8000
    
    # Compute power spectrum
    f, pxx = welch(signal, fs=fs, nperseg=1024)
    
    # Extract harmonics frequency range
    harmonic_range = np.where((f >= 70) & (f <= 4500))[0]
    
    # Extract noise frequency range
    noise_range = np.where((f >= 1500) & (f <= 4500))[0]
    
    # Compute power in harmonics range and noise range
    ph = np.sum(pxx[harmonic_range])
    pn = np.sum(pxx[noise_range])
    
    # Compute NHR
    nhr = pn / ph
    
    return nhr

In [13]:
def voice_turbulence_index(file_path):
    
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    fs = 8000
    
    # Set filter coefficients
    b = np.array([1, -1])
    a = np.array([1, -0.99])
    
    # Filter the signal to extract the high-frequency component
    high_freq = lfilter(b, a, signal)
    
    # Calculate the L1 norm of the high-frequency component
    l1_norm = np.sum(np.abs(high_freq)) / len(high_freq)
    
    # Calculate the VTI in decibels (dB)
    vti = l1_norm
    
    return vti

In [14]:
def soft_pronunciation_index(file_path):
    
    # Load the DAT file
    signal = np.fromfile(file_path, dtype=np.int16)

    # Scale the data to between -1 and 1
    signal = signal / np.max(np.abs(signal))

    # Define the sampling frequency
    fs = 8000
    
    # Compute the energy in the first formant band (500-1000 Hz)
    f1_band = [500, 1000]
    nfft = int(np.power(2, np.ceil(np.log2(len(signal)))))
    f, psd = welch(signal, fs, window='hamming', nperseg=nfft, scaling='spectrum')
    psd_f1_band = np.sum(psd[(f >= f1_band[0]) & (f <= f1_band[1])])

    # Compute the total energy
    psd_total = np.sum(psd)

    # Compute SPI as the ratio of energy in the first formant band to the total energy
    spi = (psd_f1_band / psd_total)

    return spi

In [17]:
if __name__ == "__main__":
    path_csv = "C:/Users/Rohit/mtp_pathologocal_speech"
    audio_path = "C:/Users/Rohit/mtp_pathologocal_speech/voice-icar-federico-ii-database-1.0.0"
    fundamental_freq = []
    fundamental_freq_std = []
    jitter = []
    RAP = []
    f0_variation = []
    ppq = []
    sppq = []
    Shimmer = []
    rsdva = []
    apq = []
    sapq = []
    nhr = []
    vti = []
    spi = []
    
    file = pd.read_csv(path_csv+"/info.csv")
    recordings = file["Records"]
    for f in recordings:
        p = os.path.join(audio_path,f)
        fname = p+".dat"
        
        # Calculating pitch
        pitch = extract_pitch(fname)
        fundamental_freq.append(pitch[0])
        
        # Calculating standard deviation of fundamental frequency
        var  = sum(pow(x-pitch[0],2) for x in pitch) / len(pitch)  # variance
        std  = np.sqrt(var)  # standard deviation
        fundamental_freq_std.append(std)
        
        # Calculating Jitter of speech signal
        jitter.append(extract_jitter(pitch))
        
        # Calculating RAP of speech signal
        RAP.append(calculate_rap(1/pitch))
        
        # Calculating f0 variation of speech signal
        f0_variation.append(calculate_f0_variation(fname))
        
        # Calculating PPQ of speech signal
        ppq.append(pitch_period_perturbation_quotient(fname))
        
        # Calculating SPPQ of speech signal
        sppq.append(smoothed_pitch_period_perturbation_quotient(fname))
        
        # Calculating Shimmer
        Shimmer.append(shimmer(fname))
        
        # Calculating relative standard deviation of voice amplitude
        rsdva.append(relative_standard_dev_voice_amp(fname))
        
        # Calculating amplitude perturbation quotient
        apq.append(amplitude_perturbation_quotient(fname))
        
        # Calculating smoothed amplitude perturbation quotient
        sapq.append(smoothed_amplitude_perturbation_quotient(fname))
        
        # Calculating noise to harmonic ratio
        nhr.append(noise_to_harmonic_ratio(fname))
        
        # Calculating voice turbulence index
        vti.append(voice_turbulence_index(fname))
        
        # Calculating soft pronunciation index
        spi.append(soft_pronunciation_index(fname))
        
    file["fundamental_freq"] = fundamental_freq
    file["fundamental_freq_std"] = fundamental_freq_std
    file["Jitter"] = jitter
    file["RAP"] = RAP
    file["f0_variation"] = f0_variation
    file["PPQ"] = ppq
    file["SPPQ"] = sppq
    file["Shimmer"] = Shimmer
    file["RSDVA"] = rsdva
    file["APQ"] = apq
    file["SAPQ"] = sapq
    file["NHR"] = nhr
    file["VTI"] = vti
    file["SPI"] = spi
    file.to_csv(path_csv+"/info.csv",index=False)

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.7049866  -0.12109375  0.7049866  ... -0.19204712 -0.6422119
 -0.17565918] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\159243010.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  signal = signal.astype(np.float)
C:\Users\Rohit\anaconda3\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 131072 is greater than input length  = 76160, using nperseg = 76160
  warnings.warn('nperseg = {0:d} is greater than 

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.20962524 -0.19503784 -0.20962524 ... -0.30026245 -0.58947754
 -0.06552124] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\anaconda3\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 131072 is greater than input length  = 76800, using nperseg = 76800
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.5396118  -0.4326477   0.5396118  ... -0.43737793 -0.1675415
 -0.39282227] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.2892456  -0.1842041  -0.2892456  ... -0.36746216 -0.3187

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.42269897 -0.2951355  -0.42269897 ...  0.30020142  0.60043335
  0.21508789] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.54904175 -0.24432373  0.54904175 ... -0.6594238  -0.19152832
 -0.63305664] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.5406494  -0.20587158  0.5406494  ... -0.50671387 -0.27642822
 -0.744751  ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10:

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.6763916  -0.08633423  0.6763916  ...  0.76779175  0.26486206
  0.22143555] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.21859741 -0.36376953 -0.21859741 ...  0.25686646 -0.46240234
  0.7121887 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.4524536  -0.37664795 -0.4524536  ... -0.6098938  -0.5871277
 -0.51257324] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: 

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.90530396 -0.3486328   0.90530396 ... -0.0300293  -0.87783813
 -0.26397705] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.17468262  0.16348267 -0.17468262 ...  0.19818115 -0.59091187
  0.19692993] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\anaconda3\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 131072 is greater than input length  = 77440, using nperseg = 77440
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.23880005 -0.07312012  0.23880005 ... -0.07577515 -0.208

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.17929077 -0.10067749  0.17929077 ... -0.11877441 -0.6622925
  0.15133667] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.32720947 -0.14849854 -0.32720947 ...  0.12704468  0.22393799
  0.0072937 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.572052   -0.30108643 -0.572052   ... -0.09347534  0.58255005
 -0.24105835] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: 

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.39559937 -0.17471313  0.39559937 ...  0.00604248  0.936676
 -0.13513184] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.69189453 -0.01776123  0.69189453 ... -0.20285034  0.89959717
 -0.21966553] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.6974182  -0.16772461  0.6974182  ... -0.49197388  0.06808472
 -0.13076782] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: F

C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.52526855 -0.05270386 -0.52526855 ... -0.01126099 -0.12582397
 -0.2484436 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[-0.3508606  -0.15353394 -0.3508606  ... -0.18566895  0.23492432
  0.09005737] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10: FutureWarning: Pass y=[ 0.35287476 -0.19482422  0.35287476 ... -0.24813843 -0.56814575
 -0.18173218] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  pitch, mag = librosa.piptrack(data, sr=sr)
C:\Users\Rohit\AppData\Local\Temp\ipykernel_14260\2586917211.py:10:

In [2]:
sample = []

In [4]:
if __name__ == "__main__":
    path_csv = "C:/Users/Rohit/mtp_pathologocal_speech"
    audio_path = "C:/Users/Rohit/mtp_pathologocal_speech/voice-icar-federico-ii-database-1.0.0"
    
    file = pd.read_csv(path_csv+"/info.csv")
    recordings = file["Records"]
    for f in recordings:
        p = os.path.join(audio_path,f)
        fname = p+".dat"
        
        data = np.fromfile(fname, dtype=np.int16)
        
        sample.append(len(data))

In [11]:
Max = max(sample)
Min = min(sample)
Mean = np.mean(sample)
STD = np.std(sample)

In [13]:
print(Max)
print(Max/8000)

77440
9.68


In [14]:
print(Min)
print(Min/8000)

75520
9.44


In [15]:
print(Mean)
print(Mean/8000)

76107.69230769231
9.51346153846154


In [16]:
print(STD)
print(STD/8000)

264.8120914667271
0.03310151143334089
